In [4]:
from tools import *
import os
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

Préparation et chargement des graphes pour l'entraînement du GNN.

In [ ]:
def prepare_data_for_GNN(node_features_df, edges_df, target_df):
    from_nodes = edges_df['from'].values
    to_nodes = edges_df['to'].values

    node_features = node_features_df.values
    node_features_tensor = torch.tensor(node_features, dtype=torch.float32)

    y_target = target_df['assigned_cpu'].values
    y_target_tensor = torch.tensor(y_target, dtype=torch.float32)

    unique_cpus = set(y_target)  # Get unique CPUs
    print(f"Graph: {path}")
    print(f"Number of possible CPUs (classes): {len(unique_cpus)}")
    print(f"Actual CPU values (classes) in this graph: {sorted(unique_cpus)}")
    edge_index = np.array([from_nodes, to_nodes], dtype=np.int64)
    edge_index_tensor = torch.tensor(edge_index, dtype=torch.long)

    return node_features_tensor, edge_index_tensor, y_target_tensor

graph_json_dir = "graphes_JSON"  
graph_json_files = [graph_json_dir+"/"+f for f in os.listdir(graph_json_dir) if f.endswith('.json')]
graphs = []

for path in graph_json_files:
    node_features_df=extract_node_features_from_json_file(path)
    edges_df=extract_mapped_edges_from_json(path)
    target_df=extract_optimal_repartition_from_json(path)

    node_features_tensor, edge_index_tensor, y_target_tensor = prepare_data_for_GNN(node_features_df, edges_df, target_df)
    data = Data(x=node_features_tensor, edge_index=edge_index_tensor, y=y_target_tensor)
    graphs.append(data)

graph_loader = DataLoader(graphs, batch_size=4, shuffle=True)

for batch in graph_loader:
    node_features_batch = batch.x
    edge_index_batch = batch.edge_index
    target_batch = batch.y

print("finished")

Graph: graphes_JSON/Graphe(4)_features.json
Number of possible CPUs (classes): 54
Actual CPU values (classes) in this graph: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 39, 41, 42, 45, 46, 47, 48, 49, 50, 51, 52, 53, 55, 56, 57, 58, 62]


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices